In [312]:
import pathlib
import duckdb
import shutil


# In-memory DB for quick work (change to 'mydb.duckdb' to persist)
DB_PATH = pathlib.Path("adventureworks.duckdb")


def execute_commit(query):
    """
    Execute a SQL query on the DuckDB database.
    """
    with duckdb.connect(DB_PATH) as conn:
        conn.execute(query)
        conn.execute("CHECKPOINT")  # This is crucial
        conn.commit()

    # Copy the database to a new location for use in the notebook
    shutil.copy(DB_PATH, DB_PATH.with_name("adventureworks_read.duckdb"))


def read_query(query):
    """
    Read data from the DuckDB database.
    """
    with duckdb.connect(DB_PATH) as conn:
        return conn.execute(query).df()

In [313]:
df = read_query("SELECT * FROM dimdepartmentgroup")
df.columns

Index(['DepartmentGroupKey', 'ParentDepartmentGroupKey',
       'DepartmentGroupName'],
      dtype='object')

In [314]:
df[["DepartmentGroupKey", "DepartmentGroupName"]].value_counts(normalize=True)

DepartmentGroupKey  DepartmentGroupName                 
1                   Corporate                               0.142857
2                   Executive General and Administration    0.142857
3                   Inventory Management                    0.142857
4                   Manufacturing                           0.142857
5                   Quality Assurance                       0.142857
6                   Research and Development                0.142857
7                   Sales and Marketing                     0.142857
Name: proportion, dtype: float64

In [322]:
# Modify adventureworks to an obt for easy exploration
# Denormalise the profit-side tables
def subcategory_enrichment():
    # Fake up some weights in grams and sizes in cubic cms
    # for the purpose of enriching demo examples
    qstring = """
    select
        *,
        EnglishProductSubCategoryName as ProductSubCategoryName,
        case
            when ProductSubCategoryName = 'Mountain Bikes' then 2500
            when ProductSubCategoryName = 'Road Bikes' then 2000
            when ProductSubCategoryName = 'Touring Bikes' then 3000
            when ProductSubCategoryName = 'Handlebars' then 500
            when ProductSubCategoryName = 'Bottom Brackets' then 200
            when ProductSubCategoryName = 'Brakes' then 300
            when ProductSubCategoryName = 'Chains' then 400
            when ProductSubCategoryName = 'Cranksets' then 600
            when ProductSubCategoryName = 'Derailleurs' then 700
            when ProductSubCategoryName = 'Forks' then 800
            when ProductSubCategoryName = 'Headsets' then 300
            when ProductSubCategoryName = 'Mountain Frames' then 1500
            when ProductSubCategoryName = 'Pedals' then 400
            when ProductSubCategoryName = 'Road Frames' then 1400
            when ProductSubCategoryName = 'Saddles' then 250
            when ProductSubCategoryName = 'Touring Frames' then 1600
            when ProductSubCategoryName = 'Wheels' then 1200
            when ProductSubCategoryName = 'Bib-Shorts' then 200
            when ProductSubCategoryName = 'Caps' then 100
            when ProductSubCategoryName = 'Gloves' then 150
            when ProductSubCategoryName = 'Jerseys' then 300
            when ProductSubCategoryName = 'Shorts' then 250
            when ProductSubCategoryName = 'Socks' then 50
            when ProductSubCategoryName = 'Tights' then 300
            when ProductSubCategoryName = 'Vests' then 200
            when ProductSubCategoryName = 'Bike Racks' then 1500
            when ProductSubCategoryName = 'Bike Stands' then 1000
            when ProductSubCategoryName = 'Bottles and Cages' then 200
            when ProductSubCategoryName = 'Cleaners' then 1000
            when ProductSubCategoryName = 'Fenders' then 500
            when ProductSubCategoryName = 'Helmets' then 600
            when ProductSubCategoryName = 'Hydration Packs' then 400
            when ProductSubCategoryName = 'Lights' then 300
            when ProductSubCategoryName = 'Locks' then 700
            when ProductSubCategoryName = 'Panniers' then 800
            when ProductSubCategoryName = 'Pumps' then 600
            when ProductSubCategoryName = 'Tires and Tubes' then 900
            else 1000
        end as weight_synthetic,
        case
            when ProductSubCategoryName = 'Mountain Bikes' then 15000
            when ProductSubCategoryName = 'Road Bikes' then 12000
            when ProductSubCategoryName = 'Touring Bikes' then 18000
            when ProductSubCategoryName = 'Handlebars' then 500
            when ProductSubCategoryName = 'Bottom Brackets' then 200
            when ProductSubCategoryName = 'Brakes' then 300
            when ProductSubCategoryName = 'Chains' then 400
            when ProductSubCategoryName = 'Cranksets' then 600
            when ProductSubCategoryName = 'Derailleurs' then 700
            when ProductSubCategoryName = 'Forks' then 800
            when ProductSubCategoryName = 'Headsets' then 300
            when ProductSubCategoryName = 'Mountain Frames' then 15000
            when ProductSubCategoryName = 'Pedals' then 400
            when ProductSubCategoryName = 'Road Frames' then 14000
            when ProductSubCategoryName = 'Saddles' then 250
            when ProductSubCategoryName = 'Touring Frames' then 16000
            when ProductSubCategoryName = 'Wheels' then 12000
            when ProductSubCategoryName = 'Bib-Shorts' then 200
            when ProductSubCategoryName = 'Caps' then 100
            when ProductSubCategoryName = 'Gloves' then 150
            when ProductSubCategoryName = 'Jerseys' then 300
            when ProductSubCategoryName = 'Shorts' then 250
            when ProductSubCategoryName = 'Socks' then 50
            when ProductSubCategoryName = 'Tights' then 300
            when ProductSubCategoryName = 'Vests' then 200
            when ProductSubCategoryName = 'Bike Racks' then 15000
            when ProductSubCategoryName = 'Bike Stands' then 10000
            when ProductSubCategoryName = 'Bottles and Cages' then 200
            when ProductSubCategoryName = 'Cleaners' then 1000
            when ProductSubCategoryName = 'Fenders' then 500
            when ProductSubCategoryName = 'Helmets' then 600
            when ProductSubCategoryName = 'Hydration Packs' then 400
            when ProductSubCategoryName = 'Lights' then 300
            when ProductSubCategoryName = 'Locks' then 700
            when ProductSubCategoryName = 'Panniers' then 800
            when ProductSubCategoryName = 'Pumps' then 600
            when ProductSubCategoryName = 'Tires and Tubes' then 900
            else 1000
        end as size_synthetic
    from dimproductsubcategory
    """

    return qstring


def department_enrichment(dname):
    # Enrich department with group and category
    qstring = f"""CASE
        WHEN {dname} IN ('Production','Production Control','Tool Design'
            ) THEN 4  -- Manufacturing
        WHEN {dname} IN ('Quality Assurance') THEN 5  -- Quality Assurance
        WHEN {dname} IN ('Sales', 'Marketing') THEN 7  -- Sales and Marketing
        WHEN {dname} IN ('Purchasing','Document Control','Shipping and Receiving'
            ) THEN 3  -- Inventory Management
        WHEN {dname} IN ('Human Resources', 'Information Services',
            'Facilities and Maintenance','Finance'
            ) THEN 2  -- Executive General and Administration
        WHEN {dname} IN ('Engineering', 'Research and Development'
            ) THEN 6  -- Research and Development
        WHEN {dname} IN ( 'Executive') THEN 1  -- Corporate
        ELSE 2
    END"""

    return qstring


def customer_activity_enrichment(firstmonth_col, current_month_col, last_month_col):
    # Break order aggregates into reactivated, retained, new and deactivated:
    qstring = f"""case
        when {firstmonth_col} then 'new'
        when {last_month_col} = 0 and {current_month_col} > 0 then 'reactivated'
        when {last_month_col} > 0 and {current_month_col} > 0 then 'retained'
        when {last_month_col} > 0 and {current_month_col} = 0 then 'deactivated'
        else 'inactive'
    end"""
    return qstring


def create_fact_internet_sales():
    query = f"""
    CREATE OR REPLACE table int__internetsales_orders AS
    with subcategory_enriched as (
        {subcategory_enrichment()}
    ),
    enriched_customers as (
        select
            c.CustomerKey as customer_id,
            c.MaritalStatus as marital_status,
            c.HouseOwnerFlag as house_owner,
            c.CommuteDistance as commute_distance,
            c.DateFirstPurchase as first_purchase_date,
            case
                when c.NumberCarsOwned = 0 then 'no_cars'
                when c.NumberCarsOwned = 1 then 'one_car'
                when c.NumberCarsOwned >= 2 then 'twoplus_cars'
            end as cars_category,
            case
                when c.NumberChildrenAtHome = 0 then 'no_children'
                when c.NumberChildrenAtHome = 1 then 'one_child'
                when c.NumberChildrenAtHome = 2 then 'two_children'
                when c.NumberChildrenAtHome >= 3 then 'threeplus_children'
            end as children_category,
            case
                when c.YearlyIncome >=130000 then 'super'
                when c.YearlyIncome >= 90000 then 'high'
                when c.YearlyIncome >= 50000 then 'medium'
                else 'low'
            end as income_category,
            g.CountryRegionCode as country_region_code,
            g.StateProvinceCode as state_province_code,
            g.City as city,
            t.SalesTerritoryRegion as sales_territory
        from dimcustomer c
        join dimgeography g on c.GeographyKey = g.GeographyKey
        join dimsalesterritory t on g.SalesTerritoryKey = t.SalesTerritoryKey
    ),
    customer_date_cross as (
        SELECT
            c.customer_id,
            d.report_date
        FROM enriched_customers c
        CROSS JOIN int__date_spine d
        WHERE d.report_date >= date_trunc('month', c.first_purchase_date)
    ),
    aggretated_orders as (
        SELECT
            c.customer_id,
            c.report_date,
            coalesce(count(distinct i.OrderDate), 0) as active_days,
            coalesce(count(distinct i.SalesOrderNumber), 0) as order_count,
            coalesce(count(i.SalesOrderLineNumber), 0) as order_lines,
            coalesce(sum(i.OrderQuantity), 0) as order_quantity,
            coalesce(sum(i.ExtendedAmount), 0) as order_sales,
            -coalesce(sum(i.TotalProductCost), 0) as cost_production,
            -coalesce(sum(i.TaxAmt), 0) as cost_tax,
            -coalesce(sum(i.Freight), 0) as cost_freight,
            -coalesce(sum(i.DiscountAmount), 0) as cost_discount,
            coalesce(sum(psc.weight_synthetic * i.OrderQuantity), 0) as weight_synthetic,
            coalesce(sum(psc.size_synthetic * i.OrderQuantity), 0) as size_synthetic
        FROM customer_date_cross c
        left join factinternetsales i
            on
                c.customer_id = i.CustomerKey
                and c.report_date = date_trunc('month', i.OrderDate)
        left join dimproduct p on i.ProductKey = p.ProductKey
        left join subcategory_enriched psc on p.ProductSubCategoryKey = psc.ProductSubCategoryKey
        group by 1, 2
    ),
    enriched_orders as (
        select
            a.*,
            report_date = date_trunc('month', c.DateFirstPurchase)
                as _is_first_month,
            {
                customer_activity_enrichment(
                    '_is_first_month',
                    'order_count',
                    'lag(order_count) over (partition by customer_id order by report_date)'
                    )
            } as activity_category,
            (activity_category != 'inactive')::int as buyer_count,
            date_diff('month', c.BirthDate, report_date)/12 as _customer_age_years,
            case
                when _customer_age_years < 18 then 'under_18'
                when _customer_age_years >= 18 and _customer_age_years < 30 then '18_30'
                when _customer_age_years >= 30 and _customer_age_years < 50 then '30_50'
                when _customer_age_years >= 50 and _customer_age_years < 65 then '50_65'
                when _customer_age_years >= 65 then '65plus'
            end as customer_age_category
        from aggretated_orders a
        join dimcustomer c on a.customer_id = c.CustomerKey
    )
    SELECT
        e.customer_id,
        c.marital_status,
        c.house_owner,
        c.commute_distance,
        c.cars_category,
        c.children_category,
        c.income_category,
        c.country_region_code,
        c.state_province_code,
        c.city,
        c.sales_territory,
        e.report_date,
        e.activity_category,
        e.customer_age_category,
        e.buyer_count,
        e.active_days,
        e.order_count,
        e.order_lines,
        e.order_quantity,
        e.order_sales,
        e.cost_production,
        e.cost_tax,
        e.cost_freight,
        e.cost_discount,
        e.weight_synthetic,
        e.size_synthetic
        
    FROM enriched_orders e
    join enriched_customers c on e.customer_id = c.customer_id
    
    """

    execute_commit(query)


def create_fact_reseller_sales():
    query = f"""
    CREATE OR REPLACE TABLE int__resellersales_orders AS
    with subcategory_enriched as (
        {subcategory_enrichment()}
    ),
    enriched_resellers as (
        select
            r.ResellerKey as reseller_id,
            r.BusinessType as business_type,
            r.OrderFrequency as order_frequency,
            r.ProductLine as product_line,
            r.AnnualSales as annual_sales,
            r.AnnualRevenue as annual_revenue,
            g.CountryRegionCode as country_region_code,
            g.StateProvinceCode as state_province_code,
            g.City as city,
            t.SalesTerritoryRegion as sales_territory
        from dimreseller r
        join dimgeography g on r.GeographyKey = g.GeographyKey
        join dimsalesterritory t on g.SalesTerritoryKey = t.SalesTerritoryKey
    ),
    reseller_date_cross as (
        SELECT
            r.ResellerKey as reseller_id,
            d.report_date
        FROM dimreseller r
        CROSS JOIN int__date_spine d
        WHERE d.report_date >= make_date(r.FirstOrderYear, r.OrderMonth, 1)
    ),
    aggretated_orders as (
        SELECT
            r.reseller_id,
            r.report_date,
            coalesce(count(distinct i.OrderDate), 0) as active_days,
            coalesce(count(distinct i.SalesOrderNumber), 0) as order_count,
            coalesce(count(i.SalesOrderLineNumber), 0) as order_lines,
            coalesce(sum(i.OrderQuantity), 0) as order_quantity,
            coalesce(sum(i.ExtendedAmount), 0) as order_sales,
            -0.66 * coalesce(sum(i.TotalProductCost), 0) as cost_production,
            -coalesce(sum(i.TaxAmt), 0) as cost_tax,
            -coalesce(sum(i.Freight), 0) as cost_freight,
            -coalesce(sum(i.DiscountAmount), 0) as cost_discount,
            coalesce(sum(psc.weight_synthetic * i.OrderQuantity), 0) as weight_synthetic,
            coalesce(sum(psc.size_synthetic * i.OrderQuantity), 0) as size_synthetic
        FROM reseller_date_cross r
        left join factresellersales i
            on
                r.reseller_id = i.ResellerKey
                and r.report_date = date_trunc('month', i.OrderDate)
        left join dimproduct p on i.ProductKey = p.ProductKey
        left join subcategory_enriched psc on p.ProductSubCategoryKey = psc.ProductSubCategoryKey
        group by 1, 2
    ),
    enriched_orders as (
        select
            *,
            report_date = make_date(r.FirstOrderYear, r.OrderMonth, 1)
                as _is_first_month,
            {
                customer_activity_enrichment(
                    '_is_first_month',
                    'order_count',
                    'lead(order_count) over (partition by reseller_id order by report_date)'
                    )
            } as activity_category,
            (activity_category != 'inactive')::int as buyer_count
        from aggretated_orders
        join dimreseller r on aggretated_orders.reseller_id = r.ResellerKey
    )
    SELECT
        e.reseller_id,
        r.business_type,
        r.order_frequency,
        r.product_line,
        r.annual_sales,
        r.annual_revenue,
        r.country_region_code,
        r.state_province_code,
        r.city,
        r.sales_territory,
        e.report_date,
        e.activity_category,
        e.buyer_count,
        e.active_days,
        e.order_count,
        e.order_lines,
        e.order_quantity,
        e.order_sales,
        e.cost_production,
        e.cost_tax,
        e.cost_freight,
        e.cost_discount,
        e.weight_synthetic,
        e.size_synthetic
    FROM enriched_orders e
    join enriched_resellers r on e.reseller_id = r.reseller_id
    
    """

    execute_commit(query)


def create_date_spine():
    query = """
    create or replace table int__date_spine as
    with paydays as (
     SELECT CAST(RANGE AS DATE) AS payday
     from RANGE(DATE '2010-01-01', DATE '2020-12-31', interval '14 days') 
    ),
    grouped_paydays as (
        SELECT
            DATE_TRUNC('month', payday) AS month_start,
            count(*) AS payday_count
        FROM paydays
        group by 1
    ),
    calendar_features as (
        SELECT
            DATE_TRUNC('month', day) AS month_start,
            COUNT(*) AS days_in_month,
            COUNT(CASE WHEN EXTRACT(dow FROM day) BETWEEN 1 AND 5 THEN 1 END) AS workdays_in_month,
            COUNT(CASE WHEN EXTRACT(dow FROM day) IN (0, 6) THEN 1 END) AS weekends_in_month
        FROM (
            SELECT DATE '2010-01-01' + i::INT AS day
            FROM range(0, 3650) AS t(i)  -- ~10 years of dates
        )
        GROUP BY month_start
    ),
    periods as (
        SELECT
            CAST(RANGE AS DATE) AS report_date,
            DATE_ADD(RANGE, INTERVAL 1 MONTH) AS period_end
        FROM RANGE(DATE '2010-12-01', DATE '2013-12-01', INTERVAL 1 MONTH)
    )
    select
        p.report_date,
        p.period_end,
        g.payday_count,
        c.days_in_month,
        c.workdays_in_month
    from periods p
    join grouped_paydays g on g.month_start = p.report_date
    join calendar_features c on c.month_start = p.report_date
    order by p.report_date
    
    """
    execute_commit(query)


def create_department_costings():
    query = f"""
    CREATE OR REPLACE TABLE int__department_costings AS
    with department_dates as (
        select
            d.DepartmentGroupKey as department_id,
            d.DepartmentGroupName as department_name,
            s.report_date,
            s.period_end,
            s.days_in_month,
            s.workdays_in_month
        from dimdepartmentgroup d
        cross join int__date_spine s
    ),
    
    classified AS (
        SELECT 
            DATE_TRUNC('month', Date - interval 1 day) AS report_date,
            g.DepartmentGroupKey AS department_id,
            g.DepartmentGroupName AS department_name,

            a.AccountDescription AS account_description,

            CASE
            -- Blowout drivers of labor costs
            WHEN a.AccountDescription ILIKE '%Salaries%' THEN 'salary_cost'
            WHEN a.AccountDescription ILIKE '%Payroll Taxes%' THEN 'bc_cost'
            WHEN a.AccountDescription ILIKE '%Employee Benefits%' THEN 'bc_cost'
            WHEN a.AccountDescription ILIKE '%Variances%' THEN 'oc_cost'
            WHEN a.AccountDescription ILIKE '%Training%' THEN 'tc_cost'
            WHEN a.AccountDescription ILIKE '%Travel%' THEN 'trc_cost'
            WHEN a.AccountDescription ILIKE '%Misc%' or a.AccountDescription ILIKE '%Labor%' THEN 'mc_cost'
            WHEN a.AccountDescription ILIKE '%Vehicles%' THEN 'trc_cost'
            WHEN a.AccountDescription ILIKE '%Other Expenses%' THEN 'mc_cost'
 
            WHEN a.AccountDescription ILIKE '%Conferences%' THEN 'tc_cost'
            WHEN a.AccountDescription ILIKE '%Meals%' THEN 'trc_cost'

            


            -- Non-labor
            WHEN a.AccountDescription ILIKE '%Entertainment%' THEN 'gtm_cost'
            WHEN a.AccountDescription ILIKE '%Marketing%' THEN 'gtm_cost'
            WHEN a.AccountDescription ILIKE '%Advertising%' THEN 'gtm_cost'
            WHEN a.AccountDescription ILIKE '%Facilities%' THEN 'fac_cost'
            WHEN a.AccountDescription ILIKE '%Rent%' THEN 'fac_cost'
            WHEN a.AccountDescription ILIKE '%Utilities%' THEN 'fac_cost'
            WHEN a.AccountDescription ILIKE '%Telephone%' THEN 'fac_cost'
            WHEN a.AccountDescription ILIKE '%Building Leasehold%' THEN 'fac_cost'
            WHEN a.AccountDescription ILIKE '%Equipment%' THEN 'fac_cost'
            WHEN a.AccountDescription ILIKE '%Office Supplies%' THEN 'fac_cost'

            WHEN a.AccountDescription ILIKE '%IT%' THEN 'iti_cost'
            WHEN a.AccountDescription ILIKE '%Software%' THEN 'iti_cost'
            WHEN a.AccountDescription ILIKE '%Professional Services%' THEN 'fps_cost'
            WHEN a.AccountDescription ILIKE '%Legal%' THEN 'fps_cost'
            WHEN a.AccountDescription ILIKE '%Consulting%' THEN 'fps_cost'
            WHEN a.AccountDescription ILIKE '%Interest Expense%' THEN 'misc_cost'
            



            -- Statistical
            WHEN a.AccountDescription ILIKE '%Headcount%' THEN 'headcount'

            WHEN a.AccountType in ('Assets', 'Liabilities') THEN 'ignore'
            ELSE 'NULL'
        END AS metric_name,
        a.AccountType AS account_type,
            f.Amount * if(a.Operator = '-', -1, 1) * if(a.AccountType = 'Expenditures', -1, 1)
                AS value

        FROM FactFinance f
        JOIN DimAccount a ON f.AccountKey = a.AccountKey
        JOIN DimDate d ON f.DateKey = d.DateKey
        JOIN DimDepartmentGroup g ON f.DepartmentGroupKey = g.DepartmentGroupKey
        JOIN DimScenario s ON f.ScenarioKey = s.ScenarioKey
        WHERE 
            s.ScenarioName = 'Actual'
            AND a.AccountDescription IS NOT NULL
        ),
    account_entries_raw AS (
        SELECT
            d.report_date,
            d.department_name,

            coalesce(SUM(IF(metric_name = 'headcount', value, 0)), 0) AS headcount,
            0.35 *coalesce(SUM(IF(metric_name = 'salary_cost', value, 0)), 0) AS salary_cost,
            
            coalesce(SUM(IF(metric_name = 'bc_cost', value, 0)), 0) AS benefits_cost,
            0.5 * coalesce(SUM(IF(metric_name = 'oc_cost', value, 0)), 0) AS overtime_cost,
            coalesce(SUM(IF(metric_name = 'tc_cost', value, 0)), 0) AS training_cost,
            coalesce(SUM(IF(metric_name = 'trc_cost', value, 0)), 0) AS travel_cost,
            coalesce(SUM(IF(metric_name = 'mc_cost', value, 0)), 0) AS misc_labor_cost,

            coalesce(SUM(IF(metric_name = 'gtm_cost', value, 0)), 0) AS go_to_market_cost,
            -0.015 * coalesce(SUM(IF(metric_name = 'fac_cost', value, 0)), 0) AS facilities_cost,
            -0.015 * coalesce(SUM(IF(metric_name = 'iti_cost', value, 0)), 0) AS it_infra_cost,
            coalesce(SUM(IF(metric_name = 'fps_cost', value, 0)), 0) AS fees_prof_services_cost,
            -coalesce(SUM(IF(metric_name = 'misc_cost', value, 0)), 0) AS misc_nonlabor_cost,

            

            coalesce(SUM(IF(account_type = 'Expenditures', value, 0)), 0) AS total_expenditures,
            coalesce(SUM(IF(account_type = 'Revenue', value, 0)), 0) AS total_revenue

        
        FROM department_dates d
        left join classified
            on
                classified.report_date = d.report_date
                AND classified.department_id = d.department_id
        WHERE metric_name IS NOT NULL
        GROUP BY 1, 2

    )

    select * from account_entries_raw
    """

    execute_commit(query)


create_date_spine()

create_department_costings()
create_fact_reseller_sales()
create_fact_internet_sales()

In [316]:
query = """
SELECT * from int__department_costings
where department_id = 7
"""

query = """
select
    *
from int__department_costings
-- where department_name = 'Sales and Marketing'
"""
# query = """
#     select
#         report_date,
#         sum(order_sales) as total_sales
#     from int__internetsales_orders
#     group by 1
# """

df = read_query(query)
df.groupby("report_date").sum().sort_index().head(10)
# counts = df.groupby([df.metric_name, df.account_description]).value.sum()
# empty = counts.reset_index()
# empty = empty[empty.metric_name == "NULL"]

,department_name,headcount,salary_cost,benefits_cost,overtime_cost,training_cost,travel_cost,misc_labor_cost,go_to_market_cost,facilities_cost,it_infra_cost,fees_prof_services_cost,misc_nonlabor_cost,total_expenditures,total_revenue
report_date,,,,,,,,,,,,,,,
2010-12-01,Executive General and AdministrationInventory ...,125.0,-193703.4900,-73967.64,-20136.85,-854.19,-24034.86,-2889.81,-3771.77,-8341.3784,2530.6446,-3278.62,-3739.56,-826908.06,684407.29
2011-01-01,Inventory ManagementExecutive General and Admi...,121.0,-190823.8455,-73711.50,-41572.26,-882.88,-25150.14,-2933.74,-6407.90,-27075.8338,-106119.8738,-3563.72,-3958.20,-1144524.11,2142835.31
2011-02-01,ManufacturingResearch and DevelopmentSales and...,123.0,-190864.8315,-74663.40,-26974.38,-737.80,-25146.08,-3026.59,-5780.78,-29729.9722,-95737.4118,-3453.83,-4223.41,-947964.40,1585867.64
2011-03-01,Quality AssuranceResearch and DevelopmentSales...,127.0,-197595.5760,-74123.78,-36959.84,-805.39,-23719.37,-3014.13,-4595.63,-28834.2896,-88199.1580,-3237.38,-4060.80,-937245.56,1200784.93
2011-04-01,Research and DevelopmentSales and MarketingQua...,124.0,-200535.4755,-72547.72,-67604.24,-808.01,-21818.31,-2994.83,-7131.93,-64106.7166,-160948.7074,-2957.11,-3901.77,-1350881.02,3209707.53
2011-05-01,Inventory ManagementExecutive General and Admi...,127.0,-201168.0855,-77712.62,-42330.40,-854.47,-24927.66,-3030.74,-8157.20,-72755.8400,-115162.9408,-3393.66,-4021.28,-1097676.59,2310458.04
2011-06-01,Executive General and AdministrationInventory ...,126.0,-191886.2190,-78645.63,-12815.49,-818.30,-25953.02,-2925.97,-5100.75,-64289.6380,-64292.8720,-3622.28,-3949.10,-861697.43,996903.96
2011-07-01,ManufacturingQuality AssuranceResearch and Dev...,126.0,-197885.5830,-80470.88,-57722.00,-797.22,-24819.35,-2917.65,-9162.76,-87443.7342,-100975.0404,-3413.06,-3870.86,-1344858.58,2582109.18
2011-08-01,Research and DevelopmentSales and MarketingQua...,127.0,-207510.3810,-73432.51,-51087.43,-762.97,-23794.30,-3118.47,-5103.84,-93646.5610,-69127.6606,-3143.99,-4164.87,-1117056.40,2035032.14
